In [1]:
import numpy as np
from tqdm import tqdm
import copy
from Step1_mar import mar_cd,mar_iter_ols,mar_PROJ,mar_adalasso,mar_scad,mar_lasso
from Step2_vector import vector_adalasso, vector_lasso, vector_scad
from Step3_sparse_mar import final_mar_cd,final_mar_adalasso
from utils import rate,metrics_dif_kron,matrix_stack,genetate_data

In [4]:
p = 1
dim_A = 4
dim_B = 4
T = 150
A_true,B_true,X = genetate_data(dim_A,dim_B,T,p,p_1=0.5,uniform=[-1,1],uniform_x=[1,10])
# X = matrix_stack(X, p)
U1, Sigma1, V1 = np.linalg.svd(A_true)
U2, Sigma2, V2 = np.linalg.svd(B_true)
V3, Sigma3, V3 = np.linalg.svd(np.kron(B_true,A_true))
print(max(abs((Sigma1))<1))
print(max(abs((Sigma2))<1))
print(max(abs((Sigma3))<1))
print(A_true)
print(B_true)

X_train = X[:100,:,:]
X_pred = X[100:,:,:]

True
True
True
[[-0.33192698 -0.57980376  0.17405077  0.14689558]
 [-0.1981445   0.         -0.         -0.        ]
 [-0.07656045  0.63082105 -0.21828678 -0.07284609]
 [ 0.0465573  -0.          0.05999086  0.        ]]
[[-0.          0.          0.19200803 -1.18637702]
 [-0.21573649  0.60713426 -0.          0.24652368]
 [ 0.60049639  0.          0.1694448   0.        ]
 [ 0.29656647 -0.37617722 -0.         -1.03310827]]


In [5]:
A_cd, B_cd = mar_cd(X_train,p)
lambda_ = 1
A_hat_adalasso,B_hat_adalasso = vector_adalasso(X_train,A_cd,B_cd,p,lam=lambda_)
# A_hat1, B_hat1 = final_mar_cd(X_train,p,A_hat_adalasso,B_hat_adalasso,A_cd,B_cd,max_iter=500)
A_hat1, B_hat1 = final_mar_adalasso(X_train,A_hat_adalasso,B_hat_adalasso,A_cd,B_cd,p,lam1=1,lam2=1,max_iter=500)
A_hat_1, B_hat_1 = mar_adalasso(X_train,A_cd,B_cd,p,lam1=1,lam2=1,max_iter=500)
print('转换模型 adaptive lasso')
rate(A_true,B_true,A_hat_adalasso,B_hat_adalasso)
print('mar adaptive lasso')
rate(A_true,B_true,A_hat_1,B_hat_1)

 66%|██████▌   | 329/500 [00:01<00:00, 213.60it/s]

转换模型 adaptive lasso
rate_A_mar_0: 0.6
rate_B_mar_0: 0.6666666666666666
rate_A_mar_1: 0.8181818181818182
rate_B_mar_1: 1.0
rate_mar_0: 0.6363636363636364
rate_mar_1: 0.9047619047619048
mar adaptive lasso
rate_A_mar_0: 0.4
rate_B_mar_0: 0.6666666666666666
rate_A_mar_1: 0.8181818181818182
rate_B_mar_1: 1.0
rate_mar_0: 0.5454545454545454
rate_mar_1: 0.9047619047619048


In [6]:
print('张量积误差')
print('cd')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_cd[:,i*dim_A:(i+1)*dim_A],B_hat=B_cd[:,i*dim_B:(i+1)*dim_B]))
print('mar adaptive lasso')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_hat_1[:,i*dim_A:(i+1)*dim_A],B_hat=B_hat_1[:,i*dim_B:(i+1)*dim_B]))
print('多步法')
for i in range(p):
    print(metrics_dif_kron(A_true[:,i*dim_A:(i+1)*dim_A],B_true[:,i*dim_B:(i+1)*dim_B],A_hat=A_hat1[:,i*dim_A:(i+1)*dim_A],B_hat=B_hat1[:,i*dim_B:(i+1)*dim_B]))


张量积误差
cd
-1.8811752111976865
mar adaptive lasso
-1.9553017519162659
多步法
-1.9605040675235803


In [40]:
print('最小二乘估计')
pred1 = copy.deepcopy(X_pred)
pred1[0,:,:] = A_cd[:,:dim_A]@X_train[-1,:,:]@B_cd[:,:dim_B].T
for j in range(1,p):
    pred1[0,:,:] += A_cd[:,j*dim_A:(j+1)*dim_A]@X_train[-1,:,:]@B_cd[:,j*dim_B:(j+1)*dim_B].T
for i in range(1,pred1.shape[0]):
    
    pred1[i,:,:] = A_cd[:,:dim_A]@X_train[-1,:,:]@B_cd[:,:dim_B].T
    for j in range(1,p):
        pred1[i,:,:] += A_cd[:,j*dim_A:(j+1)*dim_A]@pred1[i-1,:,:]@B_cd[:,j*dim_B:(j+1)*dim_B].T
np.mean(abs(pred1-X_pred))

最小二乘估计


2.0546988675378963

In [41]:
print('矩阵自回归adaptive lasso')
pred2 = copy.deepcopy(X_pred)
# pred2[0,:,:] = A_hat_1@X_train[-1,:,:]@B_hat_1.T
# for i in range(1,pred2.shape[0]):
#     pred2[i,:,:] = A_hat_1@pred2[i-1,:,:]@B_hat_1.T
pred2[0,:,:] = A_hat_1[:,:dim_A]@X_train[-1,:,:]@B_hat_1[:,:dim_B].T
for j in range(1,p):
    pred2[0,:,:] += A_hat_1[:,j*dim_A:(j+1)*dim_A]@X_train[-1,:,:]@B_hat_1[:,j*dim_B:(j+1)*dim_B].T
for i in range(1,pred1.shape[0]):
    pred2[i,:,:] = A_hat_1[:,:dim_A]@X_train[-1,:,:]@B_hat_1[:,:dim_B].T
    for j in range(1,p):
        pred2[i,:,:] += A_hat_1[:,j*dim_A:(j+1)*dim_A]@pred2[i-1,:,:]@B_hat_1[:,j*dim_B:(j+1)*dim_B].T
np.mean(abs(pred2-X_pred))

矩阵自回归adaptive lasso


2.054440965167662

In [42]:
print('多步法')
pred3 = copy.deepcopy(X_pred)
# pred3[0,:,:] = A_hat1@X_train[-1,:,:]@B_hat1.T
# for i in range(1,pred3.shape[0]):
#     pred3[i,:,:] = A_hat1@pred3[i-1,:,:]@B_hat1.T
pred3[0,:,:] = A_hat1[:,:dim_A]@X_train[-1,:,:]@B_hat1[:,:dim_B].T
for j in range(1,p):
    pred3[0,:,:] += A_hat1[:,j*dim_A:(j+1)*dim_A]@X_train[-1,:,:]@B_hat1[:,j*dim_B:(j+1)*dim_B].T
for i in range(1,pred1.shape[0]):
    pred3[i,:,:] = A_hat1[:,:dim_A]@X_train[-1,:,:]@B_hat1[:,:dim_B].T
    for j in range(1,p):
        pred3[i,:,:] += A_hat1[:,j*dim_A:(j+1)*dim_A]@pred3[i-1,:,:]@B_hat1[:,j*dim_B:(j+1)*dim_B].T
np.mean(abs(pred3-X_pred))

多步法


2.052689622682266